# Análisis de sueldos con Python

## Data

* [2021.2 - sysarmy - Encuesta de remuneración salarial Argentina
](https://docs.google.com/spreadsheets/d/1-ZRznwS4TK74o90aOiCKS5SiXxUQ2buN1wxZIMHQmzQ/edit#gid=557755710) 
* [2021.2 - sysarmy - Encuesta de remuneración salarial Latam
](https://docs.google.com/spreadsheets/d/1BkBNt1MHVS7DeIlpgmK9l6krtSQ5t_olRhlcyxMeKy0/edit#gid=557755710)


1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué?

1. Descargar, leer y unir en una sola estructura de datos. ¿Qué estructura elegiste y por qué

In [3]:
# # Estructura de datos: Voy a utilizar una lista de diccionarios.
# # Cada item de la lista es un diccionario que tiene como claves los elementos de la fila 
# # de de títulos (preguntas de la encuesta), y como valor la respuesta.
# # La selección se basa en el hecho de que se puede iterar sobre todas las respuestas 
# # de la lista, y que la lista sea de diccionarios me permite representar, para cada 
# # respuesta, los valores correspondientes a cada pregunta.

# Acá coloco las funcones para tratamiento de los datos...

def tsv_to_list (filename: str) -> list:
    '''
    Reads file, and generates a list of strings, the lines in file
    '''
    result = list()
    # Abro el archivo separado por tabulaciones
    with open (filename) as file:     
        # Extraigo del archivo una lista con todas las filas
        for line in iter(file.readline, ''):
            # quito los salto de linea del final de las líneas...
            line = line.replace('\n','').replace("\n","")
            result.append(line)
    return result

# lines_latam = tsv_to_list('2021.2 - sysarmy - Encuesta de remuneración salarial Latam_tab.txt')
# print (lines_latam)

def get_row (lines_list:list, row_index:int, separator:str ) -> list:
    '''
    Reads a list of lines, and returns a list corresponding to row elements in this index, 
    splitted by separator
    '''
    row = lines_list[row_index].split(separator)
    return row

# headings_latam = get_row(lines_latam, 10, '\t' )
# print(headings_latam)

def line_list_to_array (line_list:list, separator:str) -> list:
    '''
    Toma una lista de lineas importadas de un csv, tsv, excel, y separa cada linea por separador, 
    devolviendo una lista de filas cuyos elementos son celdas... 
    '''
    result = []
    for line in line_list:
        result.append(line.split(separator))
    return result

# cells_latam = line_list_to_array(lines_latam,'\t' )
# print (len(cells_latam[10]),'cells')

def rename_cell (row:list, index:int, new_name:str) -> list:
    '''
    Renames element corresponding to index in list 
    '''
    result = list(row)
    result[index] = new_name
    return result

#renamed = rename_cell(headings_latam, 0, 'Nombre nuevo')
#print (renamed)

def delete_column (row_list:list, column_index:str) -> list:
    '''
    Takes array (list of lists) imported form csv, tsv, xls (rows)
    Returnes almost same array with entire column column_index deleted from array 
    
    '''
    for row in row_list:
        del row[column_index]
    return row_list
      
# a = delete_column(cells_latam,0)
# print(a)
        
# Para el último paso, cuando ya acomodè toda la lista    
def array_to_dict (array:list, headings_row:int) -> list:   
    '''
    Takes an array (list of lists) imported from a csv, tsv, xls (rows) and returns a list of dicts
    in which each key corresponds to a heading (heading_row). Ignores lines above headings_row, and assumes
    data is below headings_row.
    If heading row is empty, names key as a str , corresponding to ' Col_' + index number in list. 
    If heading already exists, modifies it, adding _Col + column index as a suffix.  
    '''
    db = []
    headings_list = array[headings_row]
    for row in array[headings_row+1:]:
        dict_line = dict() 
        data = row
        for i in range(0,len(headings_list)):      
            if headings_list[i] in dict_line.keys():
                dict_line[headings_list[i]+'_Col_'+str(i)] = data [i]
            else:
                if headings_list[i] == '':
                    dict_line['Col_'+str(i)] = data[i]
                else:
                    dict_line[headings_list[i]] = data [i]
                
        db.append(dict_line)  
    return db


#b = array_to_dict (cells_arg,9)
#print(b)
# print(len(cells_arg[45]),' cells')
# print(len(b[45]),' cells')
# print(b[0])

# # Verifico que todos los diccionarios de la lista tengan la misma longitud (cantidad de claves)
# verif_set = set()
# for item in b:
#     verif_set.add(len(item))
# print(verif_set)

def dict_maker (db:list, key:str) -> dict:
    '''
    Given a list of lists of dictionaries with same keys, and a key, returns a dictionary that counts total
    number of ocurrences for each value under that key
    
    Example:
    db = [ {key1 : value1, key2 : value2, ...}, {key1: value3, key2: value4, ...0}, ... ]

    dict_maker (db, key2) = {value2: n, value3: m, ... }
    
    being n, m,.... the total count of values under key2 in all the dictionary list db
    '''
    key_dict = dict()
    for db_dict in db:
        data = db_dict[key]
        if data in key_dict:
            key_dict[data] += 1
        else:
            key_dict[data] = 1
    return key_dict


def dict_total_count (count_dict:dict) -> int:
    '''
    Given dict containing integers or floats as values, dict_total_count adds up these values
    to return a total sum
    '''
    count = 0
    for key in count_dict:
        count += count_dict[key]
    return count

def dict_others ( dictionary:dict, thr:int, keyname = 'Others') -> dict:
    '''
    Given a dict whith values as integer counts of elements, returns same dictionary, but grouping 
    elements with values less than threshold in a key named keyname 
    '''
    result = dict()
    result[keyname] = 0
    for role in dictionary.items():
        if role[1] < thr:
            result[keyname] += role[1]
        else: 
            result[ role[0] ] = role [1]

    return result

def format_career (career:str) -> str:
    '''
    Standarizes career names:
     - Capitalizes names
     - Manages special characters
     - Changes lic or lic. to Licenciatura; tec or tec. to Tecnicatura; cs or cs. to Ciencias;
     ed, ed. to Education
     - Ver otros
    '''
    # Transformo a mimnusculas, para facilitar el analisis:
    career = career.lower()
    # Reemplazo de tildes y abreviaturas
    replacementent_list  = ( 
    ('á','a'), 
    ('é','e'), 
    ('í','i'),
    ('ó', 'o'), 
    ('ú', 'u'),
    ('lic ','licenciatura '),
    ('lic.', 'licenciatura '),
    (' tec ', 'tecnicatura '),
    ('tec.', 'tecnicatura '),
    (' cs ', 'ciencias '),
    ('cs.','ciencias '),
    (' ed ','educacion '),
    ('ed.','educacion'),
    )
    for a,b in replacementent_list:
        career = career.replace(a,b).replace(a.upper(), b.upper())
    
    #Quito espacios en blanco al principio y al final
    for i in range(0,len(career)):
        if career[0] == ' ':
            career = career [1:]
        if career [-1] == ' ':
            cereer = career[0:-1]

    # Primer letra de cada palabra mayuscula
    career = career.title()
    
    return career


In [4]:
# En esta celda empecé a crear las variables, y códigos para recoger y estructurar los datos, 
# hacer verificaciones, etc. 
# Luego de probados, las diferentes secciohnes de codigo fueron convertidas en las funciones de la celda de arriba, 
# las que finalmente se usaron para ir respondiendo las preguntas. 
# No borro esta celda para tener un registro de la secuencia de pasos seguidos...

# # 1. Creacion de la estructura de datos, integrando los datos de ambos archivos
# # Creacion de una estructura de datos con informacion de las columnas
# # 2. Limpieza de datos, eliminar aquellas filas o columnas que no tienen informacion 
# # relevante
# # Modificar y formatear los datos inconsistentes, o distintas formas de decir lo mismo
# # (abreviaturas, tildes, mayusculas, etc)

# # Importatción de datos
# lines_latam = list()
# lines_argentina = list()
# # Abro el archivo separado por tabulaciones para latam.  
# with open ('2021.2 - sysarmy - Encuesta de remuneración salarial Latam_tab.txt' 
#           ) as f_latam:     
#     # Extraigo del archivo una lista con todas las filas
#     for line in iter(f_latam.readline, ''):
#         # quito los salto de linea del final de algunas líneas...
#         line = line.replace('\n','')
#         lines_latam.append(line)
             
# # Abro el archivo tsv para argentina
# with open ('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina_tab.txt'
#          ) as f_argentina:
#     # Extraigo del archivo una lista con todas las filas
#     for lines in iter(f_argentina.readline, ''):
#         # quito los salto de linea del final de algunas líneas..
#         line = line.replace('\n','')
#         lines_argentina.append(lines)
# # Ver si es necesario definir una función

# # Algunas visualizaciones para evaluar si los datos fueron bien importados
# # verif_set = set()
# # for lines in lines_argentina[9:12]:
# #     #pass
# #     #print(items)
# #     #print (lines_latam[11])
# #     #print(len(lines_latam[9].split('\t')))
# #     print(len(lines.split('\t')))
# #     verif_set.add(len(lines.split('\t')))
# # print (len(lines_argentina))
# # print (len(lines_latam))
# # print (verif_set)

# #Debug
#   # set1 = set()
#   # for i in range(0,len(lines_latam)):
#   #     set1.add(lines_latam[i].count('\t'))
#   # print(set1)

# # Creo la lista de encabezados, útil para generar el diccionario 
# headings_arg = lines_argentina[9].split('\t')
# headings_latam = lines_latam[10].split('\t')

# #quito el salto de linea al final 
# headings_latam[-1] = headings_latam[-1].replace('\n','')

# #Verifico si hay encabezados repetidos:
# headings_set_arg = set(headings_arg)
# headings_set_latam = set(headings_latam)
# print('arg', len(headings_arg), len(headings_set_arg)) #60 57
# print('latam', len (headings_latam),len(headings_set_latam)) # 78 75

# # Hay repetidos en los dos casos, los voy a encontrar
# #Para argentina
# index = -1
# for headings in headings_arg:
#     counter = 0
#     index += 1
#     for headings2 in headings_arg:
        
#         if headings == headings2:
#             counter += 1
#         if counter > 1:
#             print(headings,' esta repetido', 'index', index)
# print ('__\n')

# # Las columnas con encabezado repetido son las preguntas Salir o 
# # seguiir contestando?, índices 30, 38, 42 y 46 en arg
# #Hacer lo mismo para latam, o verificarlo en el archivo
# index = -1
# for headings in headings_latam:
#     counter = 0
#     index += 1
#     for headings2 in headings_latam:
        
#         if headings == headings2:
#             counter += 1
#         if counter > 1:
#             print(headings,' esta repetido', 'index', index)
# # Las columnas con encabezado repetido son las preguntas Salir o 
# # seguiir contestando?, índices 48, 56, 60 y 64 en arg
# #Hacer lo mismo para latam, o verificarlo en el archivo

# # Acá no se qué hacer. Lo más práctico es modificarlo en el csv (tsv), pero
# # capaz que a efectos de aprender puedo intentar modificar las claves al generar la
# # lista de strings, o al hacer el split modificar los valores de los[indices 
# # correspondientes... 
# #                   
            
# #inserto encabezado faltante
# headings_latam[43]='X' #Hay que verificar el nombre 

# # Integración de datos
# # Como primera aproximación, parecería que las columnas correspondientes a los países no
# # aportan información extra, podrían sustituirse por dos campos, uno para el país y otro 
# # para la región

# # No se si primero es mejor crear la lista de dicts y después acondicionarla, o hacerlo
# # antes... o hacerlo todo junto

# #list_argentina = list()
# #list_latam = list()

# db_arg = list()
# db_latam = list()

# #print (lines_latam[11:12])
# data =list()
# # Creo una lista cuyos elementos son diccionarios que recogen, para cada elemento, 
# # la informacion correspondiente a las columnas de la tabla. 

# # Para los datos de latam:
# for lines in lines_latam[11:]:
#     dict_latam = dict() # Ver nota (*) abajo 
#     data = lines.split('\t')
#     for i in range(0,len(data)):
#         #print (i, headings_latam[i], data[i])
#         dict_latam[headings_latam[i]] = data[i]
#         #print (dict_latam[headings_latam[i]])
#     db_latam.append(dict_latam) 

# # Para los datos de Argentina
# for lines in lines_argentina[10:]:
#     dict_arg = dict() # Ver nota (*) abajo 
#     data = lines.split('\t')
#     for i in range(0,len(data)):
#         # print(len(data)) #60
#         #print (i, headings_latam[i], data[i])
#         dict_arg[headings_arg[i]] = data[i]
#         # print (i, headings_arg[i])
#     db_arg.append(dict_arg)     
# # (*) En una primera instancia este reseteo estaba hecho fuera del for inicial
# # por lo que la lista quedaba con todos los elementos iguales. 
    
# #print('DB arg',db_arg[0:2])    
# #print('DB latam',db_latam[0:2])
# # print (len(db_latam))

# #print (db_latam[0] == db_latam[80])
# # print(db_latam[0:100])

# # Acondiciono los datos latam para eliminar las keys correspondientes a los paises, y dejar solo 
# # una clave para pais y otra para Región. 
# db_latam_clean = list()
# for rows in db_latam:
#     for i in range (1,19):
#         if rows[headings_latam[i]] == '':
#             #print(rows[headings_latam[i]])
#             rows.pop(headings_latam[i],0)
#         else:
#             rows['Region'] = rows[headings_latam[i]]
#             del (rows[headings_latam[i]])
#     db_latam_clean.append(rows)
# # En el caso de argentina, renombro las culumnas para que sean equivalentes a la 
# # base de datos de latam
# db_arg_clean = list()

# for rows in db_arg:
#   #  print ('columnas en db_arg: ', len(row)) 58
#     temp = rows['Dónde estás trabajando']
#     rows['Estoy trabajando en'] = 'Argentina'
#     rows['Region']  = temp
#     del (rows['Dónde estás trabajando'])
#     db_arg_clean.append(rows)


# #print(db_latam_clean[0:1])
# #print(db_arg_clean[0:1])
# print (len(db_latam_clean[0]))
# print (len(db_arg_clean[0]))
# # Hay que ver por qu[e quedaron 58 encabezados, si eran 60 columnas en el caso de arg
# # 78 en latam, a las que le quitamos 18...

# print (len(db_latam[1]))
# print (len(db_arg[2]))

# print (len(headings_arg))
# print(len(headings_latam))

# # Ahora que estan las dos bases de datos, vamos a comparar los encabezados, 
# # para ver si son consistrentes
# # arg_clean_keys = set(db_arg_clean[0].keys())
# # latam_clean_keys = set(db_latam_clean[0].keys())
# # print (arg_clean_keys,'\n',latam_clean_keys)
# # print(arg_clean_keys == latam_clean_keys)
# # print (len(arg_clean_keys))
# # print (len(latam_clean_keys))
# # verify1 = dict()
# # verify2 = dict()
# # for key in arg_clean_keys:
# #     if key in latam_clean_keys:
# #         verify1 [key] = 1
# # for key in latam_clean_keys:
# #     if key in arg_clean_keys:
# #         verify2 [key] = 1
        
# # print(verify1)
# # print(verify2)
# # verify1 == verify2

# # Ahora integro ambas listas 
# db_complete = db_arg_clean + db_latam_clean

# #print (db_complete[0],'\n', db_complete[-1])
# print(len(db_complete))
# # Cantidad de datos consistentes (quito 11 en latam y 10 en argentina)
# # Total de filas 6450 + 440 = 6890
# # 6890 - 21 = 6869, igual al  largo de la base de datos...

# # Pendiente depurar este código y ver de modularizar si aplica
# # 

# # # Debug
# # for row in db_complete:
# #      print (row['Estoy trabajando en'],': ',row['Region'])

In [5]:
# Esta celda es copia de la de arriba, pero voy a ir sustituyendo las lineas
# de codigo por las funciones creadas mas arriba

# 1. Creacion de la estructura de datos, integrando los datos de ambos archivos
# Creacion de una estructura de datos con informacion de las columnas
# 2. Limpieza de datos, eliminar aquellas filas o columnas que no tienen informacion 
# relevante
# Modificar y formatear los datos inconsistentes, o distintas formas de decir lo mismo
# (abreviaturas, tildes, mayusculas, etc)


# Al trabajar con archivos csv, dado que el separador es una coma, y a veces el texto
# de las respuestas contiene comas, al hacer el split para separar columnas, dichas
# comas introducian nuevos campos, generando inconsistencias con la cantidad de columnas de 
# datos. Opté por exportar el archivo a "tsv" (tab separated values),con lo que obtuve 
# longitudes de lista consistentes con la cantidad de columnas del archivo de
# datos original

#Algunas constantes
headings_latam_index = 10
headings_arg_index = 9

#Importaciòn de datos:
lines_latam = tsv_to_list('2021.2 - sysarmy - Encuesta de remuneración salarial Latam_tab.txt')
lines_arg = tsv_to_list('2021.2 - sysarmy - Encuesta de remuneración salarial Argentina_tab.txt')

#Creo la lista de listas (array de celdas)
cells_latam = line_list_to_array (lines_latam,'\t')
cells_arg = line_list_to_array(lines_arg,'\t')

# Creo la lista de diccionarios cuyas claves corresponden a cada pregunta de cada una de las
# encuestas
#b = array_to_dict (cells_latam,10)

db_latam = array_to_dict(cells_latam, headings_latam_index)
db_arg = array_to_dict (cells_arg, headings_arg_index)

# Integración de datos
# Como primera aproximación, parecería que las columnas correspondientes en el archivo de LAtam 
# no aportan información extra, podrían sustituirse por dos campos, uno para el país y otro 
# para la región

# Acondiciono los datos latam para eliminar las keys correspondientes a los paises, y dejar solo 
# una clave para pais y otra para Región. 

#Obtengo los encabezados de cada una de las bbdd
headings_latam = cells_latam[headings_latam_index]
headings_arg = cells_arg[headings_arg_index]

# Ojo que estos encabezados no matchean con las claves. Puede traer problemas... Resuelto mas abajo

# Si el value correspondiente al paìs està vacìo, elimino la clave.
# Si tiene algún valor, se lo asigno a la nueva clave ´Region'
# Renombro las columnas para que sean equivalentes a la base de datos de Argentina
db_latam_clean = list()
for rows in db_latam:
    for i in range (1,19):
        if rows[headings_latam[i]] == '':
            rows.pop(headings_latam[i],0)
        else:
            rows['Region'] = rows[headings_latam[i]]
            del (rows[headings_latam[i]])
    rows['Trabajo para una empresa que no tiene oficina en mi ciudad'] = rows['Col_43']
    del(rows['Col_43'])
    rows['¿Salir o seguir contestando?_1'] = rows['¿Salir o seguir contestando?_Col_56']
    rows['¿Salir o seguir contestando?_2'] = rows['¿Salir o seguir contestando?_Col_60']
    rows['¿Salir o seguir contestando?_3'] = rows['¿Salir o seguir contestando?_Col_64']
    del rows['¿Salir o seguir contestando?_Col_56']
    del rows['¿Salir o seguir contestando?_Col_60']
    del rows['¿Salir o seguir contestando?_Col_64']
    db_latam_clean.append(rows)

# En el caso de argentina, renombro las culumnas para que sean equivalentes a la 
# base de datos de latam. 
db_arg_clean = list()
for rows in db_arg:
  #  print ('columnas en db_arg: ', len(row)) 58
    rows['Estoy trabajando en'] = 'Argentina'
    rows['Region']  = rows['Dónde estás trabajando']
    del (rows['Dónde estás trabajando'])
    rows['¿Salir o seguir contestando?_1'] = rows['¿Salir o seguir contestando?_Col_38']
    rows['¿Salir o seguir contestando?_2'] = rows['¿Salir o seguir contestando?_Col_42']
    rows['¿Salir o seguir contestando?_3'] = rows['¿Salir o seguir contestando?_Col_46']
    del rows['¿Salir o seguir contestando?_Col_38']
    del rows['¿Salir o seguir contestando?_Col_42']
    del rows['¿Salir o seguir contestando?_Col_46']
    db_arg_clean.append(rows)
    
# #Debug
# #print(db_arg_clean[0:100])
# set_arg = set()
# for items in db_arg_clean:
#     set_arg.add(len(items))
    
# set_latam = set()
# for items in db_latam_clean:
#     set_latam.add(len(items))

# print (set_arg)
# print (set_latam)
    
# # Ahora integro ambas listas 
db_complete = db_arg_clean + db_latam_clean

# Testing and debug
a_set = set()
for i,row in enumerate(db_complete):
    if row['Estoy trabajando en'] == 'Honduras':
        print (i)
        a_set.add(row['Region'])        
print(a_set)
b_set = set()
for i,row in enumerate(db_complete):
    if row['Estoy trabajando en'] == 'Guatemala':
        print (i)
        b_set.add(row['Region'])  
print(b_set)
# print(db_complete[6609])
# print(db_complete[6608])

# Hay dos lineas de datos en las que la ciudad en la respuesta es incongruente con el pais:
# Esto trae problemas a posteriori.
# A efectos de resolverlos: 
# En la encuesta 6609, el pais de la respuesta es Honduras, y la ciudad Mexico DF, que pertenece a Mexico.
# Se asume que el pais correcto es el correspondiente a la region seleccionada
db_complete[6609]['Estoy trabajando en'] = 'México'
# En la encuesta 6608, el pais de respuesta es Guatemala, y la Region San Pedro Sula, que 
# pertenece a Honduras.
# En este caso, dado que es la unica respuesta que corresponde a Honduras, asumimos que el 
# pais correcto es Honduras:
db_complete[6608]['Estoy trabajando en'] = 'Honduras'

# a = list(db_complete[0].keys())
# b = list( db_complete[-1].keys())

# print('lena:',len(a), ' lenb:', len(b))
# print (set(a).difference(set(b)),'a-b')
# print (set(b).difference(set(a)),'b-a')

# print (a,'\n',b)

# count = 0
# for items in db_complete:
#     if items['Region']:
#         count += 1
# print(count)
# print(len(db_complete))

# a.sort()    
# b.sort()
# for i in range (1,61):
#     print (a[i],'|' ,b[i])

    
# #pbint (db_complete[0],'\n', -1b_complete[-1])

# # Cantidad de datos consistentes (quito 11 en latam y 10 en argentina)
# # Total de filas 6450 + 440 = 6890
# # 6890 - 21 = 6869, igual al  largo de la base de datos...

# # # Debug
# # for row in db_complete:
# #      print (row['Estoy trabajando en'],': ',row['Region'])

6609
{'México DF'}
6599
6600
6601
6602
6603
6604
6605
6606
6607
6608
{'Ciudad Guatemala', 'Antigua Guatemala', 'San Pedro Sula'}


In [6]:
# print(len(db_complete))
# print(db_complete[0:2] )


**Algunas observaciones de los datos:**
- La fila de titulos (cantidad de columnas) de la base de datos de latam tiene una longitud de 78, coincide con el conteo realizado en el excel, que tiene 78 campos útiles.
- La fila de titulos (cantidad de columnas) de la base de datos de argentina tiene una longitud de 60. Coincide con el conteo realizado en el excel, que tiene 60 campos útiles.
- La diferencia de cantidad de columnas se explica por la presencia de columnas adicionales para cada país en el archivo latam, que no están en el arg. Son exactamente 18 columnas.
- La variable lines_argentina tiene 6450 líneas, coincidente con el total de filas del excel
- La variable lines_latam tiene 440 líneas, también coincidente con el total de filas del excel
- Los datos "útiles" de la variable para argentina empiezan a partir de la fila 11  (Fila 10 (index = 9) es la de encabezados de columna), y en la variable para latam, a partir de la fila 12 , siendo la 11 (index = 10 )la de encabezados de columnas







2. Printear porcentaje de participación por región, ordenado de mayor a menor. Ej: 

```
- Ciudad Autónoma de Buenos Aires - 59.4%
- Formosa - 0.04%
```

In [7]:
# Para cada pais, creo un diccionario con las diferentes regiones, y la cantidad de 
# respuestas para esta region.

countries_set = set()
for response in db_complete:
    countries_set.add(response ['Estoy trabajando en'])

#La estructura de datos sería de este estilo
'''[{'Argentina':
  { 'Formosa':1700,
    'Chaco': 643,} 
  }, {'Venezuela':''...'}']
'''
by_country_list = list()      
countries_dict = dict()

for country in countries_set:
    countries_dict[country] = {}
    total_country_count = 0
    for response in db_complete:
        if response['Estoy trabajando en'] == country:
            total_country_count += 1
            if response['Region'] in countries_dict[country].keys():
                countries_dict[country][response['Region']] += 1
            else:
                countries_dict[country][response['Region']] = 1
    countries_dict[country]['Total'] = total_country_count

# #Debug
# print(countries_dict['Honduras'])
# print(countries_dict['Uruguay'])
# print(countries_dict['Argentina'])

# #Verif
# counter = 0
# for item in countries_dict['Argentina']:
#     counter += countries_dict['Argentina'][item]
#print(countries_dict)    
# print(counter/2) # Sumo el total tambien
for country in sorted(countries_dict):
    print(f"*{country}*")
    suma = 0
    # Ordenado por porcentaje, decreciente
    for region in sorted(countries_dict[country].items(), key = lambda item: item[1],reverse = 1):
    # Ordenado alfabeticamente por region
    #for region in sorted(countries_dict[country].items(), key = lambda item: item[0]):
        if not region[0] == 'Total':
            print( f"{region[0]:35} {countries_dict[country][region[0]] / countries_dict[country]['Total']:.2%}")            
            suma += countries_dict[country][region[0]]
       # else:
    print(f"Verif: {suma / countries_dict[country]['Total']:36.2%}") 
    print('\n')

*Argentina*
Ciudad Autónoma de Buenos Aires     55.16%
Provincia de Buenos Aires           10.78%
GBA                                 9.81%
Córdoba                             8.32%
Santa Fe                            6.21%
Mendoza                             2.33%
Entre Ríos                          1.02%
Río Negro                           0.81%
Tucumán                             0.81%
Neuquén                             0.73%
Misiones                            0.65%
Chaco                               0.61%
Salta                               0.45%
Corrientes                          0.37%
San Juan                            0.36%
Chubut                              0.30%
Jujuy                               0.30%
La Rioja                            0.25%
San Luis                            0.19%
Tierra del Fuego                    0.19%
La Pampa                            0.12%
Santiago del Estero                 0.12%
Santa Cruz                          0.06%
Catamarca           

3. Printear porcentaje de participación por tipo de rol, ordenado de mayor a menor. Ej:  
`- Developer - 39.11 %`

In [8]:
# for x in range(0,len(db_complete)): print(db_complete[x]['Trabajo de'])

# Igual que en la asignacion anterior, voy a hacer un dict que tenga las ocurrencias por tipo de rol, 
# A la hora de presentar  los resultados, los tipos de rol que contengan menos de n ocurrencias
# n = 1? se agruparan bajo el concepto otros.
# Se informaran las respuestas vacias

job_set = set()
for x in range(0,len(db_complete)): 
    #if '/' in db_complete[x]['Trabajo de']:
        job_set.add(db_complete[x]['Trabajo de'])
        
# Creo el diccionario que, para cada rol, asigna la cantidad de ocurrencias
role_dict = dict()
for response in db_complete:
    
    role = response['Trabajo de']
    if role in role_dict:
        role_dict[role] += 1
    else:
        role_dict[role] = 1

#print (role_dict,'\n')
#print (sorted(role_dict.items(), key = lambda item: item[1], reverse = True))

# #Verif
# total = 0
# for value in role_dict.values():
#     total += value
# #print (total)
            
# Defino un valor de umbral de roles, por debajo del cual agrupo los roles bajo la 
# categorìa 'Others'
umbral = 50
# Creo un segundo diccionario, que agrupa bajo la key "otros" aquellos roles que 
# tienen un conteo menor que umbral
role_dict_presentation = dict()
role_dict_presentation['Others'] = 0
for role in role_dict.items():
    if role[1] < umbral:
        role_dict_presentation['Others'] += role[1]
    else: 
        role_dict_presentation[ role[0] ] = role [1]

# Creo la funcion dict_other (movida al principio)

role_dict_presentation = dict_others (role_dict,umbral)
#print(role_dict_presentation)

print(total)
# Suma de todos los values categorizados por rol
total = dict_total_count(role_dict_presentation)

# total = 0
# for value in role_dict_presentation.values():
#     total += value

# Presentacion de los resultados
suma = 0
print('\033[1m Porcentajes de respuestas por rol\n \033[0m')
for role in sorted(role_dict_presentation.items(), key = lambda item: item[1],reverse = 1):
    if not role[0] == 'Others':
        print(f"{role[0]:35} {role[1] / total:.2%}")
        suma += role[1]
print(f"\nOthers:                             {role_dict_presentation['Others']/total:.2%}")

print(f"Verif: {(suma + role_dict_presentation['Others'])/ total:36.2%}")
        
#print('\n')    


NameError: name 'total' is not defined

In [9]:
# Voy a creqr una funciòn que me genere los diccionarios del estilo de conteo de ocurrencia
# de diferentes claves
# Funcion dict_maker creada y movida arriba, a la celda que contiene todas las funciones

# Voy a hacer una funcion a efectos de verificacion, que cuente la totalidad de elementos 
# de un diccionario de conteo
# funcion dict_total_count, creada y movida arriba

# Testing:
a = dict_maker (db_complete,'Estoy trabajando en')
print (a)
b = dict_maker (db_complete,'Trabajo de')
print (b)
# # Error si la clave no existe
# c = dict_maker(db_complete,'inexistent key')
# print(c)

print(dict_total_count(a))
print(dict_total_count(b))

{'Argentina': 6440, 'Bolivia': 10, 'Chile': 48, 'Colombia': 60, 'Costa Rica': 7, 'Cuba': 2, 'Ecuador': 30, 'El Salvador': 2, 'Guatemala': 9, 'Honduras': 1, 'México': 75, 'Nicaragua': 3, 'Panamá': 1, 'Paraguay': 8, 'Perú': 35, 'Puerto Rico': 1, 'República Dominicana': 1, 'Uruguay': 123, 'Venezuela': 13}
{'Developer': 2795, 'Technical Leader': 423, 'Networking': 106, 'Recruiter / HR': 85, 'QA / Tester': 295, 'Technical Business Analyst': 1, 'Designer': 44, 'Administrador Infraestructura': 1, 'Data Scientist': 102, 'BI Analyst / Data Analyst': 168, 'HelpDesk': 180, 'Consultant': 132, 'DBA': 49, 'help desk + backup + vmware + sys admin': 1, 'Manager / Director': 294, 'SysAdmin / DevOps / SRE': 762, 'Technical Writer': 11, 'Infosec': 98, 'Product Manager': 83, 'Data Visualization': 1, 'Data Engineer': 121, 'Mainframe': 1, 'Business Analyst': 116, 'machine learning egineer': 1, 'UX': 103, 'Project Manager': 212, 'Storage / Backup': 8, 'exploit writer': 1, 'Sales / Pre-Sales': 29, 'BI Sr. Man

4. Printear porcentaje de respuesta para las preguntas: 
* ¿Contribuís a proyectos Open Source?
* ¿Programás por hobbie?

In [10]:
#print(db_complete[0])
# Creamos el diccionario
open_source_dict = dict_maker(db_complete, '¿Contribuís a proyectos open source?')
is_hobbie_dict = dict_maker(db_complete, '¿Programás como hobbie?')
#print(open_source_dict)
#3396 + 520 + 2953
#print(is_hobbie_dict)
#3396 + 1947 + 1526
print ("Porcentaje de respuestas a la pregunta '¿Contribuís a proyectos open source?' sobre el total:")

for key in sorted(open_source_dict.items(), key = lambda item: item[1],reverse = 1):
    if key[0] == '':
            print(f"{'No contesta':<35} {key[1] / total:.2%}")
    else:
        print(f"{key[0]:35} {key[1] / total:.2%}")

print ("\nPorcentaje de respuestas a la pregunta '¿Programás como hobbie?' sobre el total:")

for key in sorted(is_hobbie_dict.items(), key = lambda item: item[1],reverse = 1):
    if key[0] == '':
            print(f"{'No contesta':<35} {key[1] / total:.2%}")
    else:
        print(f"{key[0]:35} {key[1] / total:.2%}")




Porcentaje de respuestas a la pregunta '¿Contribuís a proyectos open source?' sobre el total:


NameError: name 'total' is not defined

5. Printear porcentaje por seniority según años de experiencia, el mapeo es:
```
Junior: de 0 hasta 2 años.
Semi-Senior: de 2 años inclusive hasta 5 años.
Senior: desde 5 años inclusive.
``` 

In [11]:
# Tengo dos caminos, armar el diccionario para las respuestas dadas, y despues filtrar por el 
# criterio de seniority dado, o ya crear el diccionario filtrado. 
# Empiezo por el primer camino, porque ya tengo la funcion creada, y además, me permite visualizar
# todas las respuestas
#print(db_complete[0])
experience_dict = dict_maker (db_complete, 'Años de experiencia')

# #Verificar cantidad
# count = 0
# for years in experience_dict:
#     count += experience_dict[years]
# print(count)
   
#print(experience_dict)

# Ahora me creo un diccionario filtrado, de acuerdo a los valores de los keys
seniority_dict = {'Junior':0, 'Semi-Senior':0, 'Senior':0}
category_dict = {'Junior': range(0,2), 'Semi-Senior': range(2,5),'Senior': range(5,1000)}

for years in experience_dict:
    for key in category_dict:
        if int(float((years))) in category_dict[key]: # Ver nota (*)
            seniority_dict[key] += experience_dict[years]
    
#print(seniority_dict)
#1094 + 1829 + 3946
# (*) Originalmente habia puesto esa linea como if float(years) in category_dict[key]:
# pero me consideraba los valores float como no incluidos en el rango  (lo que está bien,
# ya que el rango es una lista de enteros), y no los incluía en la suma. 
# al aplicar la función int() al float me devuelve su perte entera, lo que me permite
# igualmente determinar si el float está en el intervalo.

# Ahora presento los datos:
print ("Distribución de seniority(*) de los encuestados:\n")
for key in seniority_dict:
    print(f"{key:35}         {seniority_dict[key] / total:.2%}")
print("\n(*) Junior: de 0 hasta 2 años.\n \
   Semi-Senior: de 2 años inclusive hasta 5 años.\n \
   Senior: desde 5 años inclusive.")

Distribución de seniority(*) de los encuestados:



NameError: name 'total' is not defined

6. Printear porcentaje de personas encuestadas por años en la compañía actual y por años en el puesto actual.

In [12]:
current_job_years_dict = dict_maker (db_complete, 'Años en el puesto actual' )
current_company_years_dict = dict_maker (db_complete, 'Años en la empresa actual' )

#Para los años en el puesto actual
print ("Antigüedad en el puesto actual de los encuestados:")
#Ordenado por valores
#for items in sorted(current_job_years_dict.items(), key = lambda item: float(item[0]), reverse = 0):
# Ordenado por claves
for items in sorted(current_job_years_dict.items(), key = lambda item: item[1], reverse = 1):
        print(f"{items[0]:35}{items[1] / total:.2%}")

# Acá también vamos a hacer una categorización por rangos, igual que en el ejemplo de arriba
cjob_years_dict = {'0-2':0, '2-5':0, '5-10':0, '10-20':0, 'Over 20':0}
year_range_dict = {'0-2': range(0,2), '2-5':range(2,5), '5-10':range(5,10), '10-20':range(10,20), 'Over 20':range(20,10000)}

for years in current_job_years_dict:
    for key in cjob_years_dict:
        if int(float((years))) in year_range_dict[key]:
            cjob_years_dict[key] += current_job_years_dict[years]
#print (cjob_years_dict)
            
print("\nPor rangos de antigüedad:")
for items in sorted(cjob_years_dict.items(), key = lambda item: item[1], reverse = 1):
        print(f"{items[0]:35}{items[1] / total:.2%}")
            
            
#Para los años en la compañía

print ("\n\nAntigüedad en la empresa actual de los encuestados:")

#Ordenado por valores
#for items in sorted(current_company_years_dict.items(), key = lambda item: float(item[0]), reverse = 0):
# Ordenado por claves
for items in sorted(current_company_years_dict.items(), key = lambda item: item[1], reverse = 1):
        print(f"{items[0]:35}{items[1] / total:.2%}")

# Acá también vamos a hacer una categorización por rangos, igual que en el ejemplo de arriba

service_years_dict = {'0-2':0, '2-5':0, '5-10':0, '10-20':0, 'Over 20':0}

for years in current_company_years_dict:
    for key in service_years_dict:
        if int(float((years))) in year_range_dict[key]:
            service_years_dict[key] += current_company_years_dict[years]
            

# Acá también vamos a hacer una categorización por rangos, igual que en el ejemplo de arriba

service_years_dict = {'0-2':0, '2-5':0, '5-10':0, '10-20':0, 'Over 20':0}
#year_range_dict = {'0-2': range(0,2), '2-5':range(2,5), '5-10':range(5,10), '10-20':range(10,20), 'Over 20':range(20,100)}

for years in current_company_years_dict:
    for key in service_years_dict:
        if int(float((years))) in year_range_dict[key]:
            service_years_dict[key] += current_company_years_dict[years]
#print (cjob_years_dict)
            
print("\nPor rangos de antigüedad:")
for items in sorted(service_years_dict.items(), key = lambda item: item[1], reverse = 1):
        print(f"{items[0]:<35}{items[1] / total:.2%}")





Antigüedad en el puesto actual de los encuestados:


NameError: name 'total' is not defined

7. Printear porcentajes de nivel de educación formal y estado, es decir: % educación secundaria, terciaria, universitaria, postgrado, doctorado, postdoctorado, completo, incompleto y en curso para cada uno.

In [13]:
#print (db_complete[5250])
# Primero voy a imprimir porcentajes sobre total de respuestas:
# Elaboro la lista de respuestas posibles para cada campo ...
education_level_order = ['Primario', 'Secundario', 'Terciario', 'Universitario', 'Posgrado',
                        'Doctorado', 'Posdoctorado']
education_level_status = ['Completado', 'Incompleto', 'En curso']
education_level_list = list()
# ... y creo la lista combinada, para que el diccionario quede ya ordenado para 
# presentr las respuestas: 
for level in education_level_order:
    for status in education_level_status:
        education_level_list.append(level +' '+ status)

# Ahora si creo el diccionario inicializando los valores en cero
education_level_dict = dict()
for item in education_level_list:
    education_level_dict[item] = 0
        
# Ahora cuento las ocurrencias de cada combinacion de nivel y estado
for dictionary in (db_complete):
    #if not dictionary['Nivel de estudios alcanzado'] == '':
        data = dictionary['Nivel de estudios alcanzado']+' '+ dictionary ['Estado']
        if data in education_level_dict:
            education_level_dict[data] += 1
        else:
            education_level_dict[data] = 1

# print(education_level_dict)
#print(dict_total_count(education_level_dict))

# Presento los resultados:
print ('Porcentaje por nivel de formación sobre el total de las respuestas:\n')

for item in education_level_dict:
        if item == ' ': 
            concept = 'No contesta'
        else:
            concept = item
        print(f"{concept:35}{education_level_dict[item] / dict_total_count(education_level_dict):.2%}")

print('\n\n')

# Desúés, porcentajes sobre los que respondieron sobre nivel de estudios
education_level_dict.pop(' ')
print ('Porcentaje por nivel de formación sobre las respuestas sobre nivel educativo:\n')

for item in education_level_dict:
        if item == ' ': 
            concept = 'No contesta'
        else:
            concept = item
        print(f"{concept:35}{education_level_dict[item] / dict_total_count(education_level_dict):.2%}")


Porcentaje por nivel de formación sobre el total de las respuestas:

Primario Completado                0.00%
Primario Incompleto                0.01%
Primario En curso                  0.00%
Secundario Completado              2.93%
Secundario Incompleto              0.23%
Secundario En curso                0.07%
Terciario Completado               4.47%
Terciario Incompleto               1.88%
Terciario En curso                 2.40%
Universitario Completado           12.74%
Universitario Incompleto           9.03%
Universitario En curso             12.87%
Posgrado Completado                2.30%
Posgrado Incompleto                0.38%
Posgrado En curso                  0.83%
Doctorado Completado               0.19%
Doctorado Incompleto               0.03%
Doctorado En curso                 0.15%
Posdoctorado Completado            0.03%
Posdoctorado Incompleto            0.01%
Posdoctorado En curso              0.01%
No contesta                        49.44%



Porcentaje por nivel de

8. Formatear las carreras universitarias:
- Nombres capitalizados
- Reemplazar vocales con tilde por vocales sin tilde.
- Reemplazar `ñ` por `n`
- *lic*, *lic.* por Licenciatura
- *tec*, *tec.* por Tecnucatura
- *cs*, *cs.* por Ciencias
- *ed*, *ed.* por Educación
- Transformaciones que se consideren necesarias

Printear porcentaje según carrera

In [14]:
# Formateo de carreras

# Se defino la funcion format_career, movida arriba
    
# # Visualizamos la variedad de carreras, para diseñar la función y testearla
# career_set = set()
# for dictionary in db_complete:
#     career_set.add(format_career(dictionary['Carrera']))
# # print (sorted(career_set))
# # print(len(career_set))

# Voy a sustituir las carreras en la bd por las carreras formateadas
# Me creo una copia de la bbdd, para modificarla sin afectar la original
db_complete_career_f = db_complete.copy()

for dictionary in db_complete_career_f:
    dictionary['Carrera'] = format_career (dictionary['Carrera'])
    
# Creo el diccionario de conteo
career_formated_dict = dict_maker (db_complete_career_f,'Carrera')

# Y creo la categoría 'Others' donde agrupo aguellas carreras con un conteo menor a umbral
umbral = 10
career_formated_dict_others = dict_others(career_formated_dict, umbral)
#print (career_formated_dict_others)

#Contabilizo el total de respuestas relevadas, para los cálculos de porcentaje y la verificación
total = dict_total_count(career_formated_dict_others)
#print(total)
# Inicializo el conteo de los elementos categorizados (salvo 'Others')
suma = 0
print('\033[1mPorcentajes de respuestas por Carrera (Totalidad de encuestas)\n \033[0m')
for career in sorted(career_formated_dict_others.items(), key = lambda item: item[1],reverse = 1):
    if career[0] == '':
        print(f"{'No responde':50} {career[1] / total:.2%}")
    else:
        if not career[0] == 'Others':
            print(f"{career[0]:50} {career[1] / total:.2%}")
    suma += career[1]
    
print(f"\n{'Others':50} {career_formated_dict_others['Others']/total:.2%}")
print(f"Verif: {suma / total:51.2%}")

# Ahora vamos a presentar los porcentajes sobre las respuestas no vacias.
career_formated_dict_others.pop('') # Eliminamos la key vacía
total = dict_total_count(career_formated_dict_others)
suma = 0
print('\n\033[1mPorcentajes de respuestas por Carrera (Solo encuestas que responden sobre carrera) \n \033[0m')
# Ordeno por value descendente ...
for career in sorted(career_formated_dict_others.items(), key = lambda item: item[1],reverse = 1):
    # e imprimo los valores del dict salvo la clave 'Others'
    if not career[0] == 'Others':
        print(f"{career[0]:50} {career[1] / total:.2%}")
        suma += career[1] # Calculo el total de respuestas en las carreras (salvo 'Others')
#Imprimo los datos que corresponden a la clave 'Others'
print(f"\n{'Others:':50} {career_formated_dict_others['Others']/total:.2%}")

# Y la verificación, como la suma de las respuestas para carreras especificas 
# más las agrupadas bajo others, todo sobre el total de respuestas
print(f"Verif: {(suma + career_formated_dict_others['Others'])/ total:51.2%}")     

Porcentajes de respuestas por Carrera (Totalidad de encuestas)
 
No responde                                        51.87%
Ingenieria En Sistemas De Informacion              9.14%
Ingenieria En Informatica                          5.46%
Analista De Sistemas                               4.18%
Licenciatura En Sistemas De Informacion            2.72%
Licenciatura En Ciencias De La Computacion         2.72%
Licenciatura En Informatica                        2.07%
Tecnicatura En Programacion                        1.63%
Ingenieria Electronica                             1.44%
Ingenieria En Computacion                          1.35%
Diseño Grafico                                     1.19%
Tecnicatura Superior En Programacion               1.14%
Licenciatura En Administracion                     0.83%
Tecnicatura En Informatica                         0.76%
Licenciatura En Analisis De Sistemas               0.57%
Ingenieria Industrial                              0.57%
Analista Programador  

9. Printear porcentaje de respuesta a las preguntas:
- ¿Realizaste cursos de especialización?
- Si realizaste, ¿Quién pagó por los cursos de especialización?

In [15]:
#print(db_complete[0:100])
courses_dict_count = dict_maker (db_complete,"Realizaste cursos de especialización" )
#print (courses_dict_count.keys())
# print (courses_dict_count)
# print (dict_total_count(courses_dict_count))

# Veo que las respuestas incluyen la respuesta, y la informacion de quien los pago
# Despues veremos de filtrar por quien pagó

# Primero vamos a imprimir los porcentajes por respuesta
total = dict_total_count(courses_dict_count)

print('\033[1mPorcentajes por respuestas sobre cursos de especialización\n \033[0m')
for answer in sorted (courses_dict_count.items(), key = lambda item: item [1], reverse = 1):
    if answer[0] == '':
        concepto = 'No contesta'
    else: 
        concepto = answer[0]
    print(f"{concepto:60} {answer[1] / total:.2%}")

# Primero vamos a imprimir los porcentajes por respuesta sin considerar las respuestas vacias
courses_dict_count.pop('')
total = dict_total_count(courses_dict_count)

print('\033[1mPorcentajes por respuestas sobre cursos de especialización\n \033[0m')
for answer in sorted (courses_dict_count.items(), key = lambda item: item [1], reverse = 1):
    if answer[0] == '':
        concepto = 'No contesta'
    else: 
        concepto = answer[0]
    print(f"{concepto:60} {answer[1] / total:.2%}")
print('\n')    
# Vamos a descartar las respuestas ambiguas (No, Si,...), y quedarnos solo con los items
# que comienzan por Sí:
courses_dict_yes_count = dict()
for item in courses_dict_count.items():
    if item[0][1:4] == "Sí,":
        courses_dict_yes_count[item[0]] = courses_dict_count[item[0]]
# print(courses_dict_count)
# print (courses_dict_yes_count)

total = dict_total_count(courses_dict_yes_count)

print("\nDe aquellos encuestados que realizaron cursos de actualización:")
# No se por qué me quedó doble el comillado de los keys...
#courses_dict_yes_count['"Sí, los pagó un empleador"']

print(" - El {:.0%} responde que fueron pagos por el empleador".format(courses_dict_yes_count['"Sí, los pagó un empleador"'] / total))
print(" - El {:.0%} responde que se realizaron de forma particular".format(courses_dict_yes_count['"Sí, de forma particular"'] / total))
print(" - El {:.0%} responde que fueron cubiertos tanto de forma particular como por el empleador".format(courses_dict_yes_count['"Sí, de forma particular, Sí, los pagó un empleador"'] / total))


Porcentajes por respuestas sobre cursos de especialización
 
No contesta                                                  49.44%
"Sí, de forma particular"                                    25.20%
No                                                           10.55%
"Sí, de forma particular, Sí, los pagó un empleador"         10.35%
"Sí, los pagó un empleador"                                  4.29%
"No, Sí, de forma particular"                                0.13%
"No, Sí, los pagó un empleador"                              0.03%
Porcentajes por respuestas sobre cursos de especialización
 
"Sí, de forma particular"                                    49.84%
No                                                           20.88%
"Sí, de forma particular, Sí, los pagó un empleador"         20.47%
"Sí, los pagó un empleador"                                  8.49%
"No, Sí, de forma particular"                                0.26%
"No, Sí, los pagó un empleador"                              0.06%


10. Printear porcentaje por identidad de género y personas con discapacidad

In [16]:
#print(db_complete[6500])
gender_dict_count = dict_maker (db_complete,'Me identifico')

# Hay muchas respuestas que no son serias, voy a tratar de filtrar por una lista acotada 
# de respuestas, descartando el resto. 
# Bajo la etiqueta 'Varón' se agruparán las respuestas: Varón, Varon, HOMBRE, hombre, Hombre 

gender_dict_r_count = { 
    'Varón Cis': 0, 
    'Mujer Cis': 0, 
    'Varón Trans': 0, 
    'Mujer Trans': 0, 
    'No binarie': 0,
    'Prefiero no decir': 0
}

#Listado de respuestas que se sustituyen por Varon Cis
varon_list = ['Varon CIS','Varón', 'Varon','Hombre','hombre','HOMBRE']
#Listado de respuestas que se sustituyen por Mujer Cis
mujer_list = ['Mujer', 'Mujer Cis']

for gender in gender_dict_count:
    #Acondicionamiento
    if gender in varon_list:
        gender = 'Varón CIS'
    if gender in mujer_list:
        gender = 'Mujer Cis'
    #Conteo    
    if gender in gender_dict_r_count:
        gender_dict_r_count[gender] += gender_dict_count[gender]
        
#print(gender_dict_r_count)

#print(sorted(gender_dict_count.items(), key = lambda item: item[1], reverse = 1) )
# for gender in gender_dict_r_count:
#     if gender in gender_dict_count.keys():
#         print(gender)
    
print("Las respuestas se filtraron para incluir solamente las siguientes categorías:")
for gender in gender_dict_r_count.keys():
    print(gender)
total = dict_total_count(gender_dict_r_count)
print(f"\nLas respuestas filtradas totalizan {total}, de un total de {dict_total_count(gender_dict_count)} ({total / dict_total_count(gender_dict_count) :.1%})")
print("\nLos porcentajes de cada respuesta son:")
for gender in gender_dict_r_count:
    print(f"{gender:40} {gender_dict_r_count[gender] / total:.1%}")

Las respuestas se filtraron para incluir solamente las siguientes categorías:
Varón Cis
Mujer Cis
Varón Trans
Mujer Trans
No binarie
Prefiero no decir

Las respuestas filtradas totalizan 7741, de un total de 6869 (112.7%)

Los porcentajes de cada respuesta son:
Varón Cis                                66.8%
Mujer Cis                                28.7%
Varón Trans                              0.2%
Mujer Trans                              0.1%
No binarie                               0.6%
Prefiero no decir                        3.6%


11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

In [17]:
#15%2
#db_complete[0]

In [18]:
#Funciones para la resolucion del problema

def avg (a:list) -> float:
    '''
    Calculates average of elements a numeric list
    '''
    return sum(a)/len(a) 

def st_dev_f (a:list) -> float:
    '''
    Calculatas standard deviation of elements in a numeric list
    '''
    average = avg (a)
    sum_cuad_dif = 0
    for value in a:
        dif = value - average
        sum_cuad_dif += dif ** 2
    return (sum_cuad_dif / len (a)) ** 0.5

# Calculadora de medianas: 
def median (values:list) -> float:
    '''
    Returns the median of a list of values. If length of the list is odd, returns true median, 
    as the value in 'the middle' of the sorted list; if even, returns a aritmethic mean of
    the two values sharing the middle. 
    '''
    sorted_values = sorted (values)
    if len (values) % 2 == 0:
        return (sorted_values[int(len(values) / 2)] + sorted_values[int(len(values) / 2) - 1]) / 2
    else:
        return sorted_values[int(len(values) / 2)]
    

#test_list = [0,1,2,3,4,5,6,7,8,9]
#print(median(test_list))
#break

#__
# Las siguientes 3 funciones estan pensadas para eliminar los valores extremos (Outliers) de la 
# lista de cotizaciones del dolar. 
# Si bien en este ejercicio, por tratarse en su mayoria de listas cortas, es mas conveniente
# la evaluacion manual de los datos, estas funciones pueden resultar utiles para
# otras series de datos a analizar.
# Queda pendiente una cuarta funcion que elimine o sustituya outliers por el metodo de percentiles,
# y finalmente una funcion que las integre todas, y permita definir las formas de gestionar los 
# outliers mediante parametros. 
# Si bien el ejercicio pide calcular medianas, tambien pueden aplicarse estas funciones para un 
# analisis mas profundo. 

def softened_mean (in_list: list, cov_factor = 2.0) -> tuple:
    ''' Given a list of floats, calculates average and st_dev, eliminates values out of
    average +- k * st_dev, and recalculates until no more elements to remove.
    Returns "softened" list, its average and std deviation. 
    '''
    values = in_list.copy()
    values_prev = None
    while not values == values_prev:
        values_prev = values.copy()
        #Average
        average = avg(values)
        #Standard Deviation
        st_dev = st_dev_f(values)
        # Clean out values out of average +- cov_factor * sigma
        for value in values:
            if value < (average - cov_factor * st_dev) or value > (average + cov_factor * st_dev):
                values.remove(value)
    
    return (values,avg(values),st_dev_f(values))

def trimmed_mean (values: list, perc = 10) -> tuple:
    ''' Given a list of floats, eliminates values in the 'perc' extremes.
    Returns trimmed list, average and std deviation of its  values
    '''
    elements_to_trim = round( len(values) * perc / 100 )
    #trim
    trimmed = sorted(values)[elements_to_trim:(len(values) - elements_to_trim)]
    #Average
    average = avg(trimmed)
    #Standard Deviation
    st_dev = st_dev_f(trimmed)
    
    return (trimmed,average,st_dev)
                             
def winsorized_mean (values: list, n = 1) -> tuple:
    ''' Given a list of floats, changes n values in top and bottom extremes for closest value. 
    Returns winzorized list, average and std deviation of its values
    '''
    #windosorize
    w = sorted(values)
    for i in range (n,0,-1):
        w[i-1] = w [i]
        w[len(w)-i] = w[len(w)-i-1]
    #Average
    average = avg(w)
    #Standard Deviation
    st_dev = st_dev_f(w)
    # Clean out values out of average +- cov_factor * sigma
    return (w,average,st_dev)
# __

# Esta funcion se podria generalizar para cualquier columna, indicando la columna asociada 
# y el valor deseado como parámetros....
def get_country_xch_rate_list (db:list, country:str) -> list:
    '''
    Given a list of dicts db, and a Country, searches for and lists dollar exchange rates for this country
    in db. Includes only values that can be converted to floats.
    '''
    xch_rate_list = list()
    for dictionary in db:
        if dictionary['Estoy trabajando en'] == country:
            try:
                xch_rate_list.append(float(dictionary['¿Cuál fue el último valor de dólar que tomaron?']))
            
            except:
                pass
    return xch_rate_list

# # Encuentro la lista de paises
# country_set = set()
# for dictionary in db_complete:
#     country_set.add(dictionary['Estoy trabajando en'])
# #print (country_set)

def column_values_set (db:list, column:str) -> set:
    '''
    Given a Column name header as column in a db, returns a set of the different values
     under this column
    '''
    result = set()
    for dictionary in db:
        result.add (dictionary[column])
    return result

def salaries_group (db:list, group_by:str ) -> dict:
    '''
    Given a db, and a string (group_by) corresponding to a header in the dictionaries, 
    returns a dict containing, each diferent value under column group_by as keys, and
    a list of values corresponding to Salary in USD)
    '''
    #Recorremos la bbdd
    #group_by = 'Trabajo de'
    
    # Creo el conjunto de valores diferentes bajo la columna
    # __Esto capaz hay qu eponerlo en funcion aparte porque se usa en otros lugares...
    grouping_set = set() 
    for item in db:
        grouping_set.add(item[group_by])
    #__
    salary_grouped_dict = dict() 
    
    # Para cada diccionario de la bbdd
    for dictionary in db:
        # Para cada valor en el set
        for value in grouping_set:
            if value == dictionary[group_by]:
                if value in salary_grouped_dict:
                    salary_grouped_dict[value].append( dictionary['Salary (USD)'] )
                else:
                    salary_grouped_dict[value] = [dictionary['Salary (USD)']]       
    return salary_grouped_dict

def table_print (title:str, dictionary:dict):
    # Esta funcion se puede mejorar para parametrizar mas partes de la misma, pero hay librarias 
    # que ya lo hacen. P.ej tabulate, pandas
    '''
    Prints formatted table of Salarys in USD grouped in dictionary
    '''
    print (f'**SALARIO POR {title.upper()} (USD)** - Ordenado por mediana de salarios\n')
    print (f"{title.title():50} {'Mediana':20} {'Media suavizada':26}{'Casos'}")
    for item in sorted(dictionary.items(),key = lambda item: median(item[1]), reverse = 1):    
        print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1],2)[1]:27.0f}{len(item[1]):19}")
    print('\u2500' * 105,'\n')
    
    print (f'**SALARIO POR {title.upper()} (USD)** - Ordenado por cantidad\n')
    print (f"{title.title():50} {'Mediana':20} {'Media suavizada':26}{'Casos'}")
    for item in sorted(dictionary.items(),key = lambda item: len(item[1]), reverse = 1):    
        print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1])[1]:27.0f}{len(item[1]):19}")
    print('\u2500' * 105,'\n') #Horizontal rule


In [19]:
# Pasos a seguir: 
# Relevar los datos de cotizacion del dolar por pais en aquellas respuestas que lo tengan, 
# e intentar inferir una cotizacion media razonable. 
# Podemos crear una funcion que cree una nueva clave en la base de datos, bajo la cual guardaremos
# el valor del salario dolarizado ... 

country_set = column_values_set(db_complete,'Estoy trabajando en')

# Obtengo un diccionario que a cada pais le asigna la lista de dollar exchange rates levantada 
# de la db
country_xch_rate_dict = dict()
for country in countries_set:
    country_xch_rate_dict[country] = get_country_xch_rate_list(db_complete, country)

#print (country_xch_rate_dict)
# print (winsorized_mean(country_xch_rate_dict['Argentina'],30))
# print (trimmed_mean(country_xch_rate_dict['Argentina'],20))
# print (softened_mean(country_xch_rate_dict['Argentina'],2.5))
# print (median(country_xch_rate_dict['Argentina']))

# Se distinguen multiples casos. Hay paises que tienen una serie de datos mas o menos 
# cuantiosa y consistente. En esos casos, se eliminaran los outliers por alguno de los metodos
# para los que se definieron funciones, y se considerara el promedio y la desviacion estandar como
# una medida de la validez de una cotizacion. Luego, cuando se recabe la cotizacion, si la misma 
# esta dentro de valores razonables para esa media y desviacion, se toma ese valor como la
# cotizacion, de lo contrario, se toma el promedio.
# En otros casos se recurrira a tomar una cotizacion de internet para verificar que datos 
# son razonables, y se trabajara con dichos datos.

# Vamos a elaborar, apoyados con las funciones desarrolaldas, un diccionario 
# con el pais como clave, y una tupla con la media y la desviacion estandar ajustadas 
# de las cotizaciones como valores. 

country_xch_rate_stats = dict()
excluded_countries = dict()
for country in country_xch_rate_dict:
    
    if len(country_xch_rate_dict[country]) > 3:
        # para aquellos países para los que tengo mas de 3 cotizaciones
        # windsorizamos la lista por aprox. el 20% de los valores mas altos y mas bajos, 
        # con un minimo de 1 y un maximo de 30 (30 es un valor que vimos conveniente
        # analizando la lista de cotizaciones para Arg, que es la más larga])
        country_xch_rate_stats[country] = winsorized_mean(country_xch_rate_dict[country],
                                            min(int(len(country_xch_rate_dict[country])*0.2)+1,30))
    elif len(country_xch_rate_dict[country]) == 3:
        # PAra aquellos que tengan exactamente 3, aplico media suavizada (excluyo
        # los valores que estan por fuera del rango avg +- 1.4 * st_dev)
        country_xch_rate_stats[country] = softened_mean(country_xch_rate_dict[country],1.4)
        #print(country)
    else:
        # En el resto de los casos, agrego la lista a una lista de excepciones, para 
        # analizarlas caso a caso
        excluded_countries [country] = country_xch_rate_dict[country]
    
for country in country_xch_rate_stats:
    print (country, country_xch_rate_stats[country][1:])   

for country in excluded_countries:
    print ('x', country, excluded_countries[country])   
# La salida de este ultimo for: 
# x Cuba []
# x Paraguay [6630.0, 6700.0]
# x Puerto Rico []
# x Guatemala [7.75, 3100.0]
# x Nicaragua [1.0, 0.0]
# x República Dominicana []
# x Honduras [20.0]
# x El Salvador []
# x Panamá []
# Para Cuba, no se encontraron datos, y no es facil establecer una equivalencia del dolar
# Para Paraguay, los valores son razonables de acuerdo a la cotizacion actual, por lo que se tomara:
country_xch_rate_stats['Paraguay'] = ( country_xch_rate_dict['Paraguay'],
                                      avg (country_xch_rate_dict['Paraguay']),
                                      st_dev_f(country_xch_rate_dict['Paraguay'])) 
#country_xch_rate_stats['Paraguay']
# Puerto Rico usa el dolar como moneda, se establece:
country_xch_rate_stats['Puerto Rico'] = ([],1,0.1)

# En gualtemala, la cotización actual es 7.73, por lo que el valor 7.75 es razonable.
# Se establece el valor declarado con una desviacion de 10%
country_xch_rate_stats['Guatemala'] = ([],7.75,0.75)

#En Honduras, la cotizaci[on actual es 24.46, se considera razonable el valor relevado, se establece:
country_xch_rate_stats['Honduras'] = ([20.0],20,2)

#En ecuador, la moneda es el dolar. Se utilizafa factor 1,0.1]
country_xch_rate_stats['Ecuador'] = (country_xch_rate_stats['Ecuador'][0],1,0.1)

# En Nicaragua, R. Dominicana, El Salvador y Panamá no hay valores consistentes, 
# se excluyen del análisis

# for country in country_xch_rate_stats:
#     print (country, country_xch_rate_stats[country][1:])   

#Sigo trabajando en el la celda siguiente .....

# El ejercicio pide, en primera instancia: *Salario según región*, y en posteriores, 
# salario agrupado por otras columnas, por lo que parece razonable la necesidad de crear una funcion 
# de filtrado. 
# Esta funcion deberia devolver, para cada valor de la columna de filtrado, un elemento de informacion
# que contenga, al menos, Pais (para verificar la validez de la cotizacion de la moneda), salario 
# y cotizacion.

# La estructura seria un diccionario que, para cada region (o columna de agrupacion) como clave, 
# tiene como value una lista de tuplas (Pais, Salario, Cotizacion)


# salary_grouped_dict = salaries_group (db_complete, 'Me identifico')
# print (len(salary_grouped_dict['Mujer Cis']))
# salary_grouped_dict['Mujer Cis'][800:]

# break

# output1 = softened_mean ((xch_rate_list),2)                
# #print(softened_mean (a_list))
# output2 = trimmed_mean (list(xch_rate_list),20)                
# #test_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]

# output3 = winsorized_mean (xch_rate_list,2)                

# #print(country_to_view)
# #print (sorted(xch_rate_list),len(xch_rate_list),'orig\n')
# print (sorted(xch_rate_list),len(xch_rate_list),'orig')
# print(avg(xch_rate_list), st_dev_f(xch_rate_list),'\n')
# print(sorted(output1[0]), len(output[0]),'Soft')
# print(output1[1:],'\n')
# print(sorted(output2[0]), len(output[0]),'Trim')
# print(output2[1:],'\n')
# print(sorted(output3[0]), len(output[0]),'Wed')
# print(output3[1:])

# break



# print(db_complete[0],'\n\n')
# brut_salary_count_dict = dict_maker(db_complete,'Salario mensual o retiro BRUTO (en tu moneda local)')
# net_salary_count_dict = dict_maker(db_complete,'Salario mensual o retiro NETO (en tu moneda local)')
# print (brut_salary_count_dict)
# print ('\n' , net_salary_count_dict)


Bolivia (6.965, 0.004999999999999893)
Costa Rica (600.0, 0.0)
Argentina (130.27303944315543, 35.48573482277905)
Colombia (3730.0, 108.70449239413551)
Ecuador (484.0, 238.67589282996684)
Venezuela (4086666.6666666665, 63420.991968134826)
Uruguay (44.15052631578948, 0.8241133907942683)
Perú (4.0615000000000006, 0.05089449872039227)
Chile (749.6666666666666, 15.107025591499548)
México (19.834545454545456, 0.18763424945954746)
x Paraguay [6630.0, 6700.0]
x Guatemala [7.75, 3100.0]
x Panamá []
x Cuba []
x El Salvador []
x República Dominicana []
x Nicaragua [1.0, 0.0]
x Honduras []
x Puerto Rico []


In [20]:

# Ahora tenemos la base de datos y las cotizaciones. Vamos a modificar la base de datos, agrgando una 
# key (columna) que tenga el salario dolarizado, y quitando todas aquellas filas que no tengan
# respondido salario.
# EL procedimiento es, listar los diccionarios y:
# - Si existe columna de salario:
#       - Obtengo el pais, verificando que se encuentre dentro de la lista con cotizacion del dolar 
#       - Obtengo la cotizacion del dolar declarada. Si la misma existe y esta dentro del promedio
#        de cotizaciones +- 2 desviaciones, considero la cotizacion como valida. De lo contrario, 
#        uso el promedio. 
#       - Calculo el Salario en dolares, y escribo la nueva clave. 

def get_xch (value:float, country:str, country_xch_rates: dict) -> float:
    '''
    Given a value, and a country in country_xch_rate db (dictionary containig, for each country, a 
    tuple (list of exchange rates, average, and st_deviation)), validates value if it is
    within average +- 2 st_dev, or returns average exchange rate if value is not float or int.  
    If conuntry not in countr_xch_rate list, returns None
    '''
    # descarto filas de paises que no tienencotizacion
    if country not in country_xch_rates:
        return None
    avg, dev = country_xch_rates[country][1:]
    
    # Intento convertir a float el valor ingresado
    try:
        value = float(value)
    except:
        # Si no puede convertirse a float, lo dejo igual, se elimina en el paso siguiente
        pass
    # Ahora, si el valor es float
    if (type(value) == float): #or type(value) == int):
        # Inicializo booleano
        valid = False
        # Si no es cero
        if not float(value) == 0:
            # Creo un booleano que indica que el valor se considera valido
            valid = avg - 2 * dev < float(value) < avg + 2 * dev
        # Si el valor es razonable lo devuelvo
        if valid: return value
        # de lo contrario, devuelvo el promedio
        else: return avg
    else: return avg
    
# # Test
# print(get_xch (0, 'Uruguay', country_xch_rate_stats))
# print(get_xch (128, 'Uruguay', country_xch_rate_stats))
# print (country_xch_rate_stats['Uruguay'][1:])    
# print((get_xch ('hola', 'Nicaragua', country_xch_rate_stats)))

# Creo la nueva bbdd, agregando la clave 'Salary (USD)'
db_salaries_usd = list()
for rows in db_complete:
    try:
        salary = float(rows['Salario mensual o retiro BRUTO (en tu moneda local)'])
    except: 
        #print('except')
        continue    
    country = rows['Estoy trabajando en']
    xch_answer = rows['¿Cuál fue el último valor de dólar que tomaron?']
    xch_rate = get_xch (xch_answer, country, country_xch_rate_stats)
    
    if xch_rate == None:
        #print('None')
        continue
    else:
        if not salary == 0.0:
            new_row = rows.copy()
            new_row ['Salary (USD)'] = round( salary / xch_rate,0) 
            db_salaries_usd.append(new_row)
        else:
            continue

print(len(db_complete))
print(len(db_salaries_usd))

# Se descarteron 139 respuestas que no presentaban salario, cuyo salario no es convertible a float, 
# o para paises para los que no se tiene cotizacion

# #Test and debug
# print (db_salaries_usd[2843])   
# column_values_set (db_salaries_usd, 'Estoy trabajando en' )  
# print(len(column_values_set (db_salaries_usd, 'Salary (USD)' )))
# print(median(list(column_values_set (db_salaries_usd, 'Salary (USD)' ))))
# sorted(column_values_set (db_salaries_usd, 'Salary (USD)' ) )[283]
        
#Cantinuo en la celda de abajo


6869
6723


11. Salarios: calcular la mediana salarial para cada una de las siguientes categorías:
- Salario según región
- Salario por rol
- Salario por experiencia
- Salario por nivel de formación
- Salario por carrera
- Salario por tecnología
- Salario por lenguaje de programación
- Salario por género

In [21]:
# Tengo la base de datos con los salarios en USD, ahora puedo realizar el agrupamiento
# por los distintos criterios que me pide la letra
# Uso la funcion creada arriba *salaries_group*

# A estos resultados les daremos el mismo tratamiento 
# salaries_by_region = salaries_group (db_salaries_usd, 'Region')
# salaries_by_role = salaries_group (db_salaries_usd, 'Trabajo de')

# En este caso podemos ademas agregar la agrupacion por seniority de ariba
#salaries_by_experience = salaries_group (db_salaries_usd, 'Años de experiencia')

# En este caso, tenemos que aplicar el acondicionamiento de nombres de carrera usado mas arriba
# salaries_by_career = salaries_group (db_salaries_usd, 'Carrera')

#Aca hay que trabajar un poco mas para separar las tecnologias y plataformas
#salaries_by_tech = salaries_group (db_salaries_usd, 'Plataformas')
#salaries_by_lang = salaries_group (db_salaries_usd, 'Lenguajes de programación o tecnologías.')

# Aca vamos a aplicar la lista de generos creada arriba
#salaries_by_gender = salaries_group (db_salaries_usd, 'Me identifico')

#print(salaries_by_region['Catamarca'])
#print(salaries_by_role['Developer'])
#level_set = column_values_set (db_salaries_usd,'Nivel de estudios alcanzado')


In [22]:
# Salarios por región
salaries_by_region = salaries_group (db_salaries_usd, 'Region')
#print(salaries_by_region)
        
# Necesitamos una correspondencia entre regiones y paises para presentacion de datos
# Utilizamos una codigo similar a la que creo countries_dict mas arriba
#print (a_set)

countries_set = column_values_set (db_salaries_usd,'Estoy trabajando en')

# [{key:value, key2: value2, ...}, {key:value2, key2:value3, ...}, ...]

region_by_country = dict()
for row in db_salaries_usd:
    if row['Estoy trabajando en'] in region_by_country:
        region_by_country[row['Estoy trabajando en' ]].add(row['Region'])
    else:
        region_by_country[row['Estoy trabajando en']] = {row['Region'],}

#print(region_by_country)

# Ahora voy a imprimir los salarios por region para cada pais
print ('**SALARIO POR PAIS Y REGION (USD)**\n')
print (f"{'País':25}{'Region':40} {'Mediana':25} {'Media suavizada':25}{'Casos':25}")
#print(salaries_by_region)
for country in region_by_country:
    for region in region_by_country[country]:
        print (f"{country:25}{region:40}{median(salaries_by_region[region]):5.0f}{softened_mean(salaries_by_region[region])[1]:30.0f}{len(salaries_by_region[region]):20}")
    print('')

**SALARIO POR PAIS Y REGION (USD)**

País                     Region                                   Mediana                   Media suavizada          Casos                    
Argentina                La Rioja                                  790                           726                  16
Argentina                Tierra del Fuego                         1274                          1056                  12
Argentina                Misiones                                  660                           587                  41
Argentina                Ciudad Autónoma de Buenos Aires          1305                          1144                3500
Argentina                GBA                                      1154                          1058                 614
Argentina                Santiago del Estero                      1028                          1103                   8
Argentina                Santa Cruz                                844                        

In [23]:
# Salarios por Rol
salaries_by_role = salaries_group (db_salaries_usd, 'Trabajo de')
#print (salaries_by_role['Developer'])

# print ('**SALARIO POR ROL (USD)** - Ordenado por cantidad\n')

# print (f"{'Rol':50} {'Mediana':20} {'Media suavizada':26}{'Casos'}")

# for item in sorted(salaries_by_role.items(),key = lambda item: len(item[1]), reverse = 1):    
#     print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1])[1]:27.0f}{len(item[1]):19}")
#     #print (item[1])
#     #{len(salaries_by_role[item[1]]):17}")
# print('\u2500' * 105,'\n')

# print ('**SALARIO POR ROL (USD)** - Ordenado por mediana de salarios\n')

# print (f"{'Rol':50} {'Mediana':20} {'Media suavizada':26}{'Casos'}")

# for item in sorted(salaries_by_role.items(),key = lambda item: median(item[1]), reverse = 1):    
#     print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1],2)[1]:27.0f}{len(item[1]):19}")
#     #print (item[1])
#     #{len(salaries_by_role[item[1]]):17}")
# print('\u2500' * 105,'\n')

#table_print ('ROL', salaries_by_role )

# Funcion de filtrado por cantidad de respuestas
umbral = 10
salaries_by_role_filtered = dict()
for item in salaries_by_role:
    if len(salaries_by_role[item]) > umbral:
        salaries_by_role_filtered [item] = salaries_by_role[item]


sorted_list = sorted(salaries_by_role_filtered.items(), key = lambda role: median(role[1]) ,reverse = 1)
#print(sorted_list)

# Algunas lineas de codigo para un analisis posterior de los datos
# Codigo para encontrar percentil
#print (sorted_list)
role = 'Scrum Master'
for i, item in enumerate(sorted_list):
    if item[0] == role:
        index = i
percentil = int(100 * ( 1 - index / len(sorted_list)))

#print(percentil)
# Codigo para, dado el percentil indica el indice del limite superior del mismo
percentil = 75
index = int ( len(sorted_list) * (1 - percentil / 100))
#print(index)
role = sorted_list [index][0]
#print (role, percentil, index)

#print(salaries_by_role_filtered)
table_print ('ROL', salaries_by_role )
table_print ('ROL', salaries_by_role_filtered )

**SALARIO POR ROL (USD)** - Ordenado por mediana de salarios

Rol                                                Mediana              Media suavizada           Casos
Director of DevOps and IT                          10701                      10701                  1
Research Engineer                                   9211                       9211                  1
subject matter expert                               7516                       7516                  1
Security Research                                   6823                       6823                  1
Principal SE                                        5361                       5361                  1
Tech Lead / Arquitecto                              5300                       5300                  1
Lead Data Engineer                                  5066                       5066                  1
Engineering Manager                                 5000                       5000                  1
CIO       

HelpDesk                                             576                        559                171
───────────────────────────────────────────────────────────────────────────────────────────────────────── 

**SALARIO POR ROL (USD)** - Ordenado por cantidad

Rol                                                Mediana              Media suavizada           Casos
Developer                                           1190                       1041               2745
SysAdmin / DevOps / SRE                             1241                       1066                744
Technical Leader                                    1842                       1655                419
Manager / Director                                  2479                       2267                291
QA / Tester                                          998                        901                290
Project Manager                                     1689                       1598                207
HelpDesk        

In [24]:
# Salario por experiencia
# Voy a usar algo similar a los rangos de seniority creados mas arriba, pero con mas categorias

experience_dict = {'0 a 2 años':range(0,2), 
                   '2 a 5 años':range(2,5),
                   '5 a 10 años': range (5,10),
                   '10 a 20 años':range (10,20),
                   'Más de 20 años':range(20,10000) }

salaries_by_experience = salaries_group (db_salaries_usd, 'Años de experiencia')

salaries_by_experience_r = dict()
for elements in experience_dict:
    salaries_by_experience_r[elements] = []

for key in salaries_by_experience:
    for ranges in experience_dict:
        if int(float(key)) in experience_dict[ranges]:
            salaries_by_experience_r[ranges] += salaries_by_experience[key]

            
# for elements in salaries_by_experience_r:
#     print(elements+';', salaries_by_experience_r[elements],'\n')    
table_print ('experiencia', salaries_by_experience_r)

**SALARIO POR EXPERIENCIA (USD)** - Ordenado por mediana de salarios

Experiencia                                        Mediana              Media suavizada           Casos
Más de 20 años                                      1689                       1413                598
10 a 20 años                                        1676                       1421               1817
5 a 10 años                                         1456                       1288               1456
2 a 5 años                                          1075                        972               1792
0 a 2 años                                           645                        567               1060
───────────────────────────────────────────────────────────────────────────────────────────────────────── 

**SALARIO POR EXPERIENCIA (USD)** - Ordenado por cantidad

Experiencia                                        Mediana              Media suavizada           Casos
10 a 20 años                            

In [25]:
# Salarios por nivel de estudios alcanzado
print ('**SALARIO POR NIVEL DE ESTUDIOS ALCANZADO (USD)**\n')
salaries_by_education = salaries_group (db_salaries_usd, 'Nivel de estudios alcanzado')
print (f"{'Nivel de estudios':26} {'Mediana':18} {'Media suavizada':22}{'Casos'}")

for item in education_level_order:
    print(f"{item:25} {median(salaries_by_education[item]):6.0f} {softened_mean(salaries_by_education[item])[1]:22.0f}{len(salaries_by_education[item]):17}")
print('\n')

salaries_by_education.pop('')
table_print('Nivel de estudios', salaries_by_education)
# print (f"{'Nivel de estudios':24} {'Media suavizada':20} {'Casos'}")
# for item in education_level_order:
#     print(f"{item:25} {softened_mean(salaries_by_education[item])[1]:6.0f} {len(salaries_by_education[item]):17}")

**SALARIO POR NIVEL DE ESTUDIOS ALCANZADO (USD)**

Nivel de estudios          Mediana            Media suavizada       Casos
Primario                    1689                   1689                1
Secundario                  1075                    939              217
Terciario                   1075                    966              591
Universitario               1270                   1047             2325
Posgrado                    1683                   1542              235
Doctorado                   1500                   1193               24
Posdoctorado                 921                   1267                3


**SALARIO POR NIVEL DE ESTUDIOS (USD)** - Ordenado por mediana de salarios

Nivel De Estudios                                  Mediana              Media suavizada           Casos
Primario                                            1689                       1689                  1
Posgrado                                            1683                       

In [26]:
# Salario por carrera

# En este caso habria que aplicar la transformacion de nombres de carrera antes de hacer el grouping
db_salaries_usd_formatted = db_salaries_usd.copy()

for i, row in enumerate (db_salaries_usd):
    db_salaries_usd_formatted[i]['Carrera'] = format_career (row['Carrera'])

salaries_by_career = salaries_group (db_salaries_usd_formatted, ('Carrera'))    

#print(salaries_by_career)

# ELimino listado para respuestas en que no se especifica carrera
salaries_by_career.pop('')


table_print('carrera', salaries_by_career)

**SALARIO POR CARRERA (USD)** - Ordenado por mediana de salarios

Carrera                                            Mediana              Media suavizada           Casos
Licenciatura Admin                                 11514                      11514                  1
X                                                   9583                       9583                  1
Ciencias De La Comunicacion Social                  7837                       7837                  1
Turismo                                             7665                       7665                  1
N/A                                                 6000                       6000                  1
Licenciatura En Ciencias De La Comunicacion Social  4806                       4806                  2
Geologia                                            4684                       4684                  1
Tecnico En Sistemas                                 4606                       4606                  1
Ingeni

In [28]:
# Salario por tecnologia
# La letra del ejercicio pide sen un punto salario por tecnologia y en otro por lenguaje, 
# pero en realidad ambos elementos estan comprendidos en la misma pregunta, por lo que 
# asumiremos que la primera consignase trata de salarios por plataforma.

salaries_by_tech = salaries_group (db_salaries_usd, 'Plataformas')

# Este ejercicio presenta la dificultad de que las respuestas incluyen varias plataformas, 
# separadas por comas. La funcion salaries_group no nos sirve por si sola a los efectos. 

# Podemos intentar un listado que incluya, para cada plataforma listada por respuesta 
# el salario en usd

#Funcion de acondicoionamiento
def format_tech (inlet:str) -> str:
    '''
    Format platform names.....
    '''
    return inlet.replace("\"","").strip("\"").lower()

# Acondiciono los nombres de plataformas
salaries_by_tech_corr = dict()

for tech in salaries_by_tech.items():
    salaries_by_tech_corr[format_tech(tech[0])] =  tech[1]

#print (salaries_by_tech_corr)


#print(salaries_by_tech)
# Elaboro un listado de tecnologias:
tech_set = set()
for platforms in salaries_by_tech_corr:
    for tech in platforms.split(', '):
        tech_set.add(tech)

#Inicializo a lista vacia cada una de esas claves        
salaries_by_tech_split = dict()
for tech in tech_set:
    salaries_by_tech_split[tech] = []    
#print(salaries_by_tech_split)

# Agrego los salario de las claves que contienen las plataformas listadas.
for platform in salaries_by_tech_corr:
    for tech in tech_set:
        if tech in platform.split(', '):
            salaries_by_tech_split[tech] += salaries_by_tech_corr[platform]

# Quito clave nula (sin respuesta sobre plataforma)
salaries_by_tech_split.pop('')
    
#imprimimos tablas
table_print('Plataforma', salaries_by_tech_split)

**SALARIO POR PLATAFORMA (USD)** - Ordenado por mediana de salarios

Plataforma                                         Mediana              Media suavizada           Casos
oracle                                              5757                       5757                  1
netlify                                             5600                       5600                  1
 ansible                                            5300                       5300                  1
bare metal                                          5300                       5300                  1
queues                                              5171                       5171                  1
elastic search                                      5171                       5171                  1
azure devops                                        4535                       4535                  1
aws elb                                             4406                       4406                  1
lok

herramientas y tecnologías open-source y propietar     1                          1                  1
telefonia                                              1                          1                  1
manatal                                                1                          1                  1
elastix                                                0                          0                  1
qtest                                                  0                          0                  1
trello                                                 0                          0                  1
skype                                                  0                          0                  1
win 10                                                 0                          0                  1
excel y word.                                          0                          0                  1
unifi.                                                 0                 

In [31]:
# Salario por lenguaje
salaries_by_lang = salaries_group (db_salaries_usd, 'Lenguajes de programación o tecnologías.')

# Similar al agrupamiento por plataformas
# Acondiciono los nombres de los lenguajes
salaries_by_lang_corr = dict()
for lang in salaries_by_lang.items():
    salaries_by_lang_corr[format_tech(lang[0])] =  lang[1]

# Elaboro un listado de lenguajes:
lang_set = set()
for languages in salaries_by_lang_corr:
    for lang in languages.split(', '):
        lang_set.add(lang)

#Inicializo a lista vacia cada una de esas claves        
salaries_by_lang_split = dict()
for lang in lang_set:
    salaries_by_lang_split[lang] = []    

# Agrego los salario de las claves que contienen los lenguajes listadas.
for language in salaries_by_lang_corr:
    for lang in lang_set:
        if lang in language.split(', '):
            salaries_by_lang_split[lang] += salaries_by_lang_corr[language]
    
#imprimimos tablas
table_print('Lenguaje', salaries_by_lang_split)

**SALARIO POR LENGUAJE (USD)** - Ordenado por mediana de salarios

Lenguaje                                           Mediana              Media suavizada           Casos
solidity                                            6141                       4512                  3
nodejs                                              5911                       5911                  1
cython                                              4615                       4615                  1
xamarin                                             4606                       4606                  2
no escribo código en este rol                       4511                       4511                  1
assembly                                            4255                       4255                  1
ecl                                                 3454                       3454                  1
graphql                                             3232                       3232                  2
node.

yaml                                                 652                        652                  1
ninguna                                              645                        645                  1
powershell (es de scripting                          620                        620                  1
i know)                                              620                        620                  1
bash script                                          614                        614                  1
typescript                                           614                        614                  1
jython                                               614                        614                  1
arangodb                                             590                        590                  1
spss                                                 583                        583                  1
n/a                                                  557                 

In [32]:
# Salario por genero
salaries_by_gender = salaries_group (db_salaries_usd, 'Me identifico')

# Acá vamos a tener en cuenta solo las respuestas con los generos especificados 
# en el ejercicio mas arriba

gender_list = list(gender_dict_r_count.keys())

#print(salaries_by_gender)
salaries_by_gender_restricted = dict()
for row in salaries_by_gender:
    cons_gender = row
    if row in varon_list:
        cons_gender = 'Varón'
    if cons_gender in gender_list:
        #print(cons_gender)
        if cons_gender in salaries_by_gender_restricted:
           # print(salaries_by_gender[row])
            salaries_by_gender_restricted[cons_gender] += salaries_by_gender[row]
        else:
            salaries_by_gender_restricted[cons_gender] = salaries_by_gender[row]
            #print(salaries_by_gender[row])
suma = 0
table_print('género', salaries_by_gender_restricted)



**SALARIO POR GÉNERO (USD)** - Ordenado por mediana de salarios

Género                                             Mediana              Media suavizada           Casos
Varón Trans                                         1689                       1627                 13
Varón Cis                                           1305                       1127               5080
Prefiero no decir                                   1234                       1113                275
No binarie                                          1232                        944                 50
Mujer Cis                                            998                        842               1066
Mujer Trans                                          768                        603                  7
───────────────────────────────────────────────────────────────────────────────────────────────────────── 

**SALARIO POR GÉNERO (USD)** - Ordenado por cantidad

Género                                             

In [33]:
# Salarios por discapacidad
salaries_by_disability = salaries_group(db_salaries_usd, '¿Tenés algún tipo de discapacidad?')

salaries_by_disability.pop('')

#print(salaries_by_dishability)
table_print ('discapacidad',salaries_by_disability)



# Habr[ia que hacer un desglose por respuestas multiples, ]

**SALARIO POR DISCAPACIDAD (USD)** - Ordenado por mediana de salarios

Discapacidad                                       Mediana              Media suavizada           Casos
"Uso lentes, honestamente no se si califica como d 10701                      10701                  1
Espectro autista                                    8004                       8004                  1
"Visual, Daltonismo"                                6141                       6141                  1
Síndrome del pene grande                            5971                       5971                  1
"Auditiva, Mental"                                  3711                       3711                  1
ADHD                                                3583                       3583                  2
Asperger                                            2622                       2622                  1
Visceral                                            2198                       1846                  3
e

12. Printear porcentaje y salario según tipos de contrato .

In [34]:
# Porcentajes y salario segun tipos de contrato
salaries_by_contract_type = salaries_group (db_salaries_usd, 'Tipo de contrato')

#print (salaries_by_contract_type)

#table_print('Tipo de contrato', salaries_by_contract_type)

# Para conseugir los porcentajes realizamos un conteo de los elementos 
# de la listade values para cadakey del diccionario 
#print (salaries_by_contract_type)
contract_type_count_dict = dict()
total = 0
for key in salaries_by_contract_type:
    total += len (salaries_by_contract_type[key])
    if not key in contract_type_count_dict:
        contract_type_count_dict[key] = len(salaries_by_contract_type[key])
    else:
        contract_type_count_dict[key] += len(salaries_by_contract_type[key])
# print(contract_type_count_dict,'\n', total)
# print (len(db_salaries_usd))     

# Tomo el codigo de la funcion tabulate, y lo
# modifico para agregar una columna de porcentajes

print (f'**SALARIO POR TIPO DE CONTRATO (USD)** - Ordenado por mediana de salarios\n')
print (f"{'Tipo de contrato':50} {'Mediana':20} {'Media suavizada':26}{'Porcentaje'}")
for item in sorted(salaries_by_contract_type.items(), key = lambda item: median(item[1]), reverse = 1):    
    print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1],2)[1]:25.0f}{len(item[1]) / total:24.1%}")
print('\u2500' * 105,'\n')

print (f'**SALARIO POR TIPO DE CONTRATO (USD)** - Ordenado por porcentaje\n')
print (f"{'Tipo de contrato':50} {'Mediana':20} {'Media suavizada':26}{'Porcentaje'}")
for item in sorted(salaries_by_contract_type.items(),key = lambda item: len(item[1]) / total, reverse = 1):    
    print(f"{item[0][0:50]:50}{median(item[1]):6.0f}{softened_mean(item[1])[1]:25.0f}{len(item[1]) / total:24.1%}")
print('\u2500' * 105,'\n') #Horizontal rule




        #     for region in sorted(countries_dict[country].items(), key = lambda item: item[1],reverse = 1):
#     # Ordenado alfabeticamente por region
#     #for region in sorted(countries_dict[country].items(), key = lambda item: item[0]):
#         if not region[0] == 'Total':
#             print( f"{region[0]:35} {countries_dict[country][region[0]] / countries_dict[country]['Total']:.2%}")            
#             suma += countries_dict[country][region[0]]
#        # else:
#     print(f"Verif: {suma / countries_dict[country]['Total']:36.2%}")
        
#     print('\n')    
    



**SALARIO POR TIPO DE CONTRATO (USD)** - Ordenado por mediana de salarios

Tipo de contrato                                   Mediana              Media suavizada           Porcentaje
Remoto (empresa de otro país)                       3672                     3178                    7.0%
Full-Time                                           1228                     1115                   82.8%
Tercerizado (trabajo a través de consultora o agen  1036                      853                    3.0%
Freelance                                            998                      656                    2.5%
Participación societaria en una cooperativa          960                     1046                    0.3%
Part-Time                                            500                      426                    4.3%
───────────────────────────────────────────────────────────────────────────────────────────────────────── 

**SALARIO POR TIPO DE CONTRATO (USD)** - Ordenado por porcentaje

Tipo d

13. En base a los resultados obtenidos confeccionar conclusiones respecto a:

- Rol vs sueldos
- Nivel de educación alcanzada vs sueldos
- Género vs sueldos
- Tecnologías populares

1. Rol vs sueldo 

De los datos relevados, los roles que representan mayores sueldos estan relacionados con cargos
de direccion, presidencia / vicepresidenvia, gerencia y liderazgo o direccion de equipos de ingenieria .

No obstante, para gran parte de esos roles, se cuenta en la mayor parte con uno o unos pocos datos, por lo que no es adecuado sacar conclusiones estadisticas de ese muestreo.  

Si realizamos un filtrado posterior por los roles que obtuvieron mas de 10 respuestas, nos quedamos con una lista reducida de los 25 roles mas comunes, obtenemos:

- En el cuartil superior de medianas de salarios mayores, encontramos los roles de: VP / C-level (mediana de salarios 2940), Manager / Director (2479), Product Manager (1919),  Architect (1896), Technical Leader (1842) , Project Manager (1689) y Scrum Master(1535)

- En el caurtil inferior: Bi Analysy / Data Analist, QA / Tester, Networking, Recruiter / HR, Designer y HelpDesk 


Limitaciones: 
- Se listan y analizan unicamente la mediana y la media suavizada, que por si solos no indican nada sobre la variabilidad de salarios, ni tampoco sobre otras condiciones y beneficios laborales. 


2. Nivel de educacion alcanzado vs salario:

Por tener una cantidad de categorias limitada, este agrupamiento nos permite visualizar claramente algunas tendencias:

- Los niveles de educacion Primaria y PosDoc no representan informacion confiable, ya que cuentan con un numero bajo de respuestas (1 y 3)
- Se verifica una tendencia a aumento de salarios cuanto mas avanzados en nivel de estudios estan  los encuestados: la mediana de salarios de los posgrados y doctorados es superior a la de los universitarios, que a su vez es superior a los niveles secundario y terciario no universitario. 
- Como datos curiosos:la mediana de salarios de los posgraduados es superior a la de los doctorados; los nivelel terciario y secundario tienen medianas de salario similares,aunque nuevamente, en ambos casos, hay que prestar atencion a las limitaciones planteadas.



3. Genero vs sueldos

 - De acuerdo a estos datos, las categorias asociadas a varones Trans y Cis son las quetienen una mayor mediana de salarios, siendo la primera la mas numerosa con mas del 70% de las respuestas. 
 - Las categorias asociadas a mujeres tienen una mediana de salarios sensiblemente inferior a las asociadas a varones.


 Nota: Este corte puede verse afectado por las diferentes percepciones que tienen los encuestados, por sesgos y dificultades de los encuestados y por la seleccion de categorias del encuestador 
 y del analista de datos. 


4. Tecnologias populares

- Las 5 plataformas que nuclearon mayor presencia en las respuestas fueron: linux, docker, AWS, Windows Server y azure
- Los 5 lenguajes de programacion que tuvieron mayor presencia en las respuestas fueron: javascript, sql, html, python (aguante python :)) y java



14. Generar gráficos de barra para mostrar los resultados mencionados anteriormente usando strings, ej:

```
developer | -----------------------------------------------
sysadmin  | ---------------------------
QA        | ------------------
          | ..................5%.......10%...............40%


In [36]:
def category_dict_count (db:dict) -> dict:
    count_dict = dict()
    total = 0
    for key in db:
        total += len (db[key])
        if not key in count_dict:
            count_dict[key] = len(db[key])
        else:
            count_dict[key] += len(db[key])    
    return count_dict

test_db = { 'key1_recorta_texto_muy_largo': 100, 
            'key2': 75,
            'key3':50,
            'key4':25,
            'key5':10,
            'key6':60
            }

def bar_graph(db:dict, elements:int = 5, greater_up:bool = True, spread_scale:bool = True,
              character:str = '\u2588', divisions:int = 4, order_by_value = True, unit:str = '', 
             title:str = 'BAR GRAPH', show_title:bool = True):
    '''
    Prints a bar graph of a dicionary (db) which values are ints, asigning each key the bar 
    proportional to its value. 
    Select first or last n elements to show, depending of value of boolean greater_up
    Boolean spread_scale defines if max_x value is 100 or max filtered value.
    character defines the simbol for the bar
    divisions defines number of divisions to represent in axis. 
    order_by_value not yet implemented.  
    '''
    # Constantes
    data_width = 100   
    key_width = 26
    unit_t = unit[0:2]
    
    # Seleccion de datos
    data = sorted(db.items(), key = lambda key: key[1], reverse = greater_up)[0:elements]
    # Escalado
    # Defino si al valor mas grande se le asigna el ancho maximo, o a 100 se le asigna elancho 
    # maximo
    # En elprimer caso represento valores abolutos, en el segundo porcentajes
    if spread_scale == True:    
        # Represento el maximo del eje horizontal como el valor maximo de la serie de datos  
        max_x = max (data, key = lambda key: key[1])[1]
    else:
        max_x = 100
    scale = lambda x: int( x / max_x * data_width)
    
    print('\n')
    # Imprime linea superior
    print('\u2500' * (data_width + key_width ))
    # Imprime titulo
    if show_title == 0: 
        f_title = ('')
    elif show_title == 1:
        f_title = f"{title}"
    else:
        f_title = f"{title.upper()} - {'Mejores' if greater_up else 'Peores'} {min(elements,len(db))} resultados (sobre un total de {len(db)})" 
    print(f"{f_title:^128}")    
# Imprime linea bajo titulo
    print('\u2500' * (data_width + key_width ))
# Imprime la linea de cada barra
    for key in data:
        # Imprimo los encabezados, las barras correspondientes y el valor de la variable
        print (f"{key[0][:key_width-1]:{key_width}}{character * scale(key[1])} {float(key[1]):.0f}")
# Linea inferior con marcas de division
    # Primer tramo de linea de division, hasta marca del cero
    print('\u2500' *  (key_width-1) , end = '')
    # Los siguientes marcas de division seguidas por el tramo de linea inferior
    for i in range(0,divisions):
        # Caracter de marca + (largo del intervalo - 1) caracteres de linea
        print('\u253c', end = '')
        print('\u2500' * ( int((data_width - 1) / divisions) ),end = '')
    # y por ultimo la marca del 100% o valor maximo
    print('\u253c')  
# Leyendas horizontales en el eje
    # Separacion hasta el 0,
    print (' ' * (key_width-3), end = '')
    # etiquetas subsiguientes hasta 100% o valor maximo
    tag = 0
    for x in range(0,divisions + 1):
        # La etiqueta de la leyenda 
        tag = x * max_x / divisions
        tag_len = len((str(int(tag))))
        decimals = 1        
        separation = int(data_width / divisions - tag_len - 2 - len(unit_t))
        print(f"{tag:.{decimals}f}{unit_t}{'':{separation}}", end = '')
    # Separo del resto de la informacion     
    print('\n')
bar_graph(test_db, elements = 6, spread_scale = 1,unit = '%', greater_up = False, 
          title = 'Porcentajes de prueba', )
bar_graph(category_dict_count(salaries_by_role), elements = 15, spread_scale = 1, divisions = 10, 
             title = ' Cantidad de respuestas por rol', show_title = 2)



──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                     Porcentajes de prueba                                                      
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
key5                      ██████████ 10
key4                      █████████████████████████ 25
key3                      ██████████████████████████████████████████████████ 50
key6                      ████████████████████████████████████████████████████████████ 60
key2                      ███████████████████████████████████████████████████████████████████████████ 75
key1_recorta_texto_muy_la ████████████████████████████████████████████████████████████████████████████████████████████████████ 100
─────────────────────────┼────────────────────────┼────────────────────────┼────────────────────────┼─────────────

In [37]:
# Porcentajes por rol
salaries_by_role

median_salary_by_role = dict()
for key in salaries_by_role_filtered:
    median_salary_by_role[key] = softened_mean(salaries_by_role_filtered[key],1.5)[1]
bar_graph (median_salary_by_role, elements = 50, show_title = 1, title = 'MEDIAN SALARIES BY ROLE (USD)')

language_split_count= category_dict_count (salaries_by_lang_split)
bar_graph (language_split_count, show_title=2,title = 'Most popular languages (answer count including language)')



──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                 MEDIAN SALARIES BY ROLE (USD)                                                  
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
VP / C-Level              ████████████████████████████████████████████████████████████████████████████████████████████████████ 2568
Manager / Director        █████████████████████████████████████████████████████████████████████████████████ 2086
Product Manager           ███████████████████████████████████████████████████████████████████ 1745
Architect                 ████████████████████████████████████████████████████████████████ 1649
Project Manager           ███████████████████████████████████████████████████████████ 1535
Middleware                █████████████████████████████████████████████████████ 1364